# Deploy the chatglm model on Amazon SageMaker

As we have finetuned the model, next we will show you how to deploy the model on SageMaker.

In this notebook, we explore how to host a large language model on SageMaker using the [Large Model Inference](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-inference.html) container that is optimized for hosting large models using DJLServing. DJLServing is a high-performance universal model serving solution powered by the Deep Java Library (DJL) that is programming language agnostic. To learn more about DJL and DJLServing, you can refer to our recent [blog post](https://aws.amazon.com/blogs/machine-learning/deploy-large-models-on-amazon-sagemaker-using-djlserving-and-deepspeed-model-parallel-inference/).

## Create a SageMaker Model for Deployment
As a first step, we'll import the relevant libraries and configure several global variables such as the hosting image that will be used nd the S3 location of our model artifacts

In [ ]:
import sagemaker
from sagemaker.model import Model
from sagemaker import serializers, deserializers
from sagemaker import image_uris
import boto3
import os
import time
import json
import jinja2
from pathlib import Path

In [ ]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

s3_client = boto3.client("s3") # client to intreract with S3 API
sm_client = boto3.client("sagemaker")  # client to intreract with SageMaker
smr_client = boto3.client("sagemaker-runtime") # client to intreract with SageMaker Endpoints
jinja_env = jinja2.Environment() # jinja environment to generate model configuration templates

In [ ]:
# lookup the inference image uri based on our current region
djl_inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117"
)

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_cache_path = Path("./model")
local_cache_path.mkdir(exist_ok=True)

model_name = "THUDM/chatglm-6b"#

# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.model", "*.py"]

model_download_path = snapshot_download(
    repo_id=model_name,
    cache_dir=local_cache_path,
    allow_patterns=allow_patterns,
)

In [ ]:
# Get the model files path
import os
from glob import glob

local_model_path = None

paths = os.walk(r'./model')
for root, dirs, files in paths:
    for file in files:
        if file == 'config.json':
            print(os.path.join(root,file))
            local_model_path = str(os.path.join(root,file))[0:-11]
            print(local_model_path)
if local_model_path == None:
    print("Model download may failed, please check prior step!")

In [ ]:
%%script env sagemaker_default_bucket=$bucket local_model_path=$local_model_path bash

chmod +x ./s5cmd
./s5cmd sync ${local_model_path} s3://${sagemaker_default_bucket}/chatglm-6b/

rm -rf model

In [ ]:
pretrained_model_location = "s3://sagemaker-us-east-1-/chatglm-6b/"# Change to the model artifact path in S3 
print(f"Pretrained model will be downloaded from ---- > {pretrained_model_location}")

## Deploying a Large Language Model using Hugging Face Accelerate
The DJL Inference Image which we will be utilizing ships with a number of built-in inference handlers for a wide variety of tasks including:
- `text-generation`
- `question-answering`
- `text-classification`
- `token-classification`

You can refer to this [GitRepo](https://github.com/deepjavalibrary/djl-serving/tree/master/engines/python/setup/djl_python) for a list of additional handlers and available NLP Tasks. <br>
These handlers can be utilized as is without having to write any custom inference code. We simply need to create a `serving.properties` text file with our desired hosting options and package it up into a `tar.gz` artifact.

Lets take a look at the `serving.properties` file that we'll be using for our first example

In [ ]:
!mkdir accelerate_src

In [ ]:
%%writefile accelerate_src/requirements.txt
transformers==4.27.1

In [ ]:
%%writefile accelerate_src/serving.template
engine=Python
option.s3url={{ s3url }}
option.task=text-generation
option.device_map=auto

In [ ]:
# we plug in the appropriate model location into our `serving.properties` file based on the region in which this notebook is running
template = jinja_env.from_string(Path("accelerate_src/serving.template").open().read())
Path("accelerate_src/serving.properties").open("w").write(template.render(s3url=pretrained_model_location))
!pygmentize accelerate_src/serving.properties | cat -n

There are a few options specified here. Lets go through them in turn<br>
1. `engine` - specifies the engine that will be used for this workload. In this case we'll be hosting a model using the [DJL Python Engine](https://github.com/deepjavalibrary/djl-serving/tree/master/engines/python)
2. `option.entryPoint` - specifies the entrypoint code that will be used to host the model. djl_python.huggingface refers to the `huggingface.py` module from [djl_python repo](https://github.com/deepjavalibrary/djl-serving/tree/master/engines/python/setup/djl_python).  
3. `option.s3url` - specifies the location of the model files. Alternativelly an `option.model_id` option can be used instead to specifiy a model from Hugging Face Hub (e.g. `EleutherAI/gpt-j-6B`) and the model will be automatically downloaded from the Hub. The s3url approach is recommended as it allows you to host the model artifact within your own environment and enables faster deployments by utilizing optimized approach within the DJL inference container to transfer the model from S3 into the hosting instance 
4. `option.task` - This is specific to the `huggingface.py` inference handler and specifies for which task this model will be used
5. `option.device_map` - Enables layer-wise model partitioning through [Hugging Face Accelerate](https://huggingface.co/docs/accelerate/usage_guides/big_modeling#designing-a-device-map). With `option.device_map=auto`, Accelerate will determine where to put each **layer** to maximize the use of your fastest devices (GPUs) and offload the rest on the CPU, or even the hard drive if you don’t have enough GPU RAM (or CPU RAM). Even if the model is split across several devices, it will run as you would normally expect.
6. `option.load_in_8bit` - Quantizes the model weights to int8 thereby greatly reducing the memory footprint of the model from the initial FP32. See this [blog post](https://huggingface.co/blog/hf-bitsandbytes-integration) from Hugging Face for additional information 

For more information on the available options, please refer to the [SageMaker Large Model Inference Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-configuration.html)

Our initial approach here is to utilize the built-in functionality within Hugging Face Transformers to enable Large Language Model hosting. 

We place the `serving.properties` file into a tarball and upload it to S3

In [ ]:
%%writefile accelerate_src/model.py
from djl_python import Input, Output
import os
import logging
import torch
import deepspeed
import transformers
# from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline, AutoTokenizer, AutoModel


model = None
#here, we need to set the global variable batch_size according to the batch_size in the serving.properties file.
batch_size = 8

def load_model(properties):

    model_location = properties['model_dir']
    
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    
    tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)

    #for HF accelerate inference

    model = AutoModel.from_pretrained(model_location, device_map="auto", torch_dtype=torch.float16, trust_remote_code=True)
    print("----------model dtype is {0}---------".format(model.dtype))
    model.requires_grad_(False)
    model.eval()
    
    return model, tokenizer

def handle(inputs: Input) -> None:

    global model, tokenizer

    try:
        if not model:
            model,tokenizer = load_model(inputs.get_properties())

        if inputs.is_empty():
            # Model server makes an empty call to warmup the model on startup
            return None
        
        if inputs.is_batch():
            #the demo code is just suitable for single sample per client request
            bs = inputs.get_batch_size()
            logging.info(f"Dynamic batching size: {bs}.")
            batch = inputs.get_batches()
            #print(batch)
            tmp_inputs = []
            for _, item in enumerate(batch):
                tmp_item = item.get_as_json()
                tmp_inputs.append(tmp_item.get("input"))
            
            #For server side batch, we just use the custom generation parameters for single Sagemaker Endpoint.
            result = model.chat(tokenizer, tmp_inputs, batch_size = bs, max_new_tokens = 128, min_new_tokens = 128, temperature = 1.0, do_sample = True)
            
            outputs = Output()
            for i in range(len(result)):
                outputs.add(result[i], key="generate_text", batch_index=i)
            return outputs
        else:
            inputs = inputs.get_as_json()
            if not inputs.get("input"):
                return Output().add_as_json({"code":-1,"msg":"input field can't be null"})

            #input data
            data = inputs.get("input")
            params = inputs.get("params",{})

            #for pure client side batch
            if type(data) == str:
                bs = 1
            elif type(data) == list:
                if len(data) > batch_size:
                    bs = batch_size
                else:
                    bs = len(data)
            else:
                return Output().add_as_json({"code":-1,"msg": "input has wrong type"})
                
            print("client side batch size is ", bs)
            #predictor
#             result = model.chat(tokenizer, data, batch_size = bs, **params)
            result = model.chat(tokenizer, data, **params)


            #return
            return Output().add({"code":0,"msg":"ok","data":result})
    except Exception as e:
        return Output().add_as_json({"code":-1,"msg":e})

In [ ]:
!tar czvf acc_model.tar.gz accelerate_src/ 

In [ ]:
s3_code_prefix = "chatglm/deploy/code"

code_artifact = sess.upload_data("acc_model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {code_artifact}")

## Deploy Model to a SageMaker Endpoint
With a helper function we can now deploy our endpoint and invoke it with some sample inputs

In [ ]:
def deploy_model(image_uri, model_data, role, endpoint_name, instance_type, sagemaker_session):
    
    """Helper function to create the SageMaker Endpoint resources and return a predictor"""
    model = Model(
            image_uri=image_uri, 
              model_data=model_data, 
              role=role
             )
    
    model.deploy(
        initial_instance_count=1,
        instance_type=instance_type,
        endpoint_name=endpoint_name,
        container_startup_health_check_timeout=60*15
        )
    
    # our requests and responses will be in json format so we specify the serializer and the deserializer
    predictor = sagemaker.Predictor(
        endpoint_name=endpoint_name, 
        sagemaker_session=sagemaker_session, 
        serializer=serializers.JSONSerializer(), 
        deserializer=deserializers.JSONDeserializer())
    
    return predictor

In [ ]:
# creates a unique endpoint name
endpoint_name = sagemaker.utils.name_from_base("chatglm-6B")
print(f"Our endpoint will be called {endpoint_name}")

In [ ]:
# deployment will take about 10 minutes
predictor = deploy_model(image_uri=djl_inference_image_uri, 
                            model_data=code_artifact, 
                            role=role, 
                            endpoint_name=endpoint_name, 
                            instance_type="ml.g4dn.xlarge", 
                            sagemaker_session=sess)

Let's run an example with a basic text generation prompt Large model inference is

In [ ]:
%%time
predictor.predict({ 
                    "input" : "请介绍下你自己", 
                    "params": { "max_length": 50, "temperature": 0.5 }
                })

Finally Let's do a quick benchmark to see what kind of latency we can expect from this model

In [ ]:
%%timeit -n3 -r1
predictor.predict({ 
                    "input" : "请介绍下你自己", 
                    "params": { "max_length": 50, "temperature": 0.5 }
                })

In [ ]:
# Clean up the endpoint before proceeding
predictor.delete_endpoint()

-------

[sagemaker-hosting/Large-Language-Model-Hosting/](https://github.com/aws-samples/sagemaker-hosting/tree/main/Large-Language-Model-Hosting)